## Vegatables: Image Classification

Dataset Source: https://www.kaggle.com/datasets/misrakahmed/vegetable-image-dataset

##### Install Necessary Libraries Not Already Installed

In [1]:
%pip install datasets transformers tensorboard evaluate

Note: you may need to restart the kernel to use updated packages.


##### Import Necessary Libraries

In [2]:
import os, sys, random
# 设置环境变量，禁用 tokenizers 库的并行处理。
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# 导入用于图像处理和绘制的库。
from PIL import ImageDraw, ImageFont, Image
import PIL.Image

# 导入 tqdm，用于在循环或迭代过程中显示进度条。
from tqdm import tqdm

# 导入 numpy 和 pandas，用于数值计算和数据操作。
import numpy as np
import pandas as pd

# 导入 datasets 库，用于处理数据集和加载数据。
import datasets
from datasets import load_dataset, Image, load_metric

# 导入 transformers 库，用于使用预训练模型和相关工具。
import transformers
from transformers import Trainer, TrainingArguments
from transformers import ViTForImageClassification, ViTFeatureExtractor

# 导入 PyTorch，用于深度学习操作。
import torch

# 导入 evaluate 库，用于模型评估指标。
import evaluate


##### Display Versions of Relevant Libraries

In [3]:
print("Python:".rjust(15), sys.version[0:6])
print("NumPy:".rjust(15), np.__version__)
print("Pandas:".rjust(15), pd.__version__)
print("Datasets:".rjust(15), datasets.__version__)
print("Transformers:".rjust(15), transformers.__version__)
print("Torch:".rjust(15), torch.__version__)

        Python: 3.9.19
         NumPy: 1.26.4
        Pandas: 1.2.4
      Datasets: 2.18.0
  Transformers: 4.39.1
         Torch: 2.2.1


##### Ingest Dataset

In [4]:
# 加载 imagefolder 数据集。指定数据集的路径和是否丢弃标签。
dataset = load_dataset("imagefolder", 
                        data_dir="../Data/fruit-and-vegetable-image-recognition", 
                        drop_labels=False)

# 打印训练数据集的信息。
print("Training Dataset")
print(dataset['train'])       # 打印训练集的整体信息。
print(dataset['train'][0])    # 打印训练集的第一个样本。
print(dataset['train'][-1])   # 打印训练集的最后一个样本。

# 打印测试数据集的信息。
print("Testing Dataset")
print(dataset['test'])        # 打印测试集的整体信息。
print(dataset['test'][0])     # 打印测试集的第一个样本。
print(dataset['test'][-1])    # 打印测试集的最后一个样本。


Resolving data files:   0%|          | 0/3115 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/351 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/359 [00:00<?, ?it/s]

Training Dataset
Dataset({
    features: ['image', 'label'],
    num_rows: 3115
})
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2218x2216 at 0x74E48BB0FA60>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1200x1806 at 0x74E48BB0F850>, 'label': 35}
Testing Dataset
Dataset({
    features: ['image', 'label'],
    num_rows: 359
})
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2218x2216 at 0x74E48BB0FE80>, 'label': 0}
{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1170 at 0x74E48BB229A0>, 'label': 35}


##### Display Grid of Examples From Each Class to Gain Better Picture of Data

In [5]:
#这个函数的目的是在数据集的每个类别中显示几个示例图片，
#并在每个图片上标注其对应的类别标签。这对于可视化和理解数据集的内容非常有用。
def show_grid_of_examples(ds, 
                          seed: int = 42, 
                          examples_per_class: int = 3, 
                          size=(350, 350)):
    '''
    该函数在数据集的每个类别中显示几个示例图片。
    '''
    w, h = size  # 设置每个图片的宽度和高度。
    labels = ds['train'].features['label'].names  # 获取所有类别的标签名称。
    grid = PIL.Image.new(mode='RGB', size=(examples_per_class * w, len(labels) * h))  # 创建一个新的空白图像用于放置所有示例图片。
    draw = ImageDraw.Draw(grid)  # 创建一个用于绘制文本的对象。
    font = ImageFont.truetype("MiSans-Normal.ttf", 24)  # 设置文本的字体和大小。
    
    for label_id, label in enumerate(labels):  # 遍历每个类别的标签。
        # 过滤出单个标签的数据集，打乱它，然后选取一些样本。
        ds_slice = ds['train'] \
                    .filter(lambda ex: ex['label'] == label_id) \
                    .shuffle(seed) \
                    .select(range(examples_per_class))
        
        # 在一行中绘制这个标签的示例图片。
        for i, example in enumerate(ds_slice):
            image = example['image']  # 获取图片。
            idx = examples_per_class * label_id + i  # 计算图片在网格中的位置。
            box = (idx % examples_per_class * w, idx // examples_per_class * h)  # 计算图片在网格中的坐标。
            grid.paste(image.resize(size), box=box)  # 将图片粘贴到网格中。
            draw.text(box, label, (255, 255, 255), font=font, dill=(0,0,255,1.0))  # 在图片上绘制标签文本。
    
    return grid  # 返回包含所有示例图片的网格图像。


In [6]:
#显示数据集中每个类别的几个示例图片
#show_grid_of_examples(dataset, seed=42, examples_per_class=3)

##### Remember to Install git lfs & Enter HuggingFace Access Token

In [7]:
# Enter Huggingface Access Token

!git lfs install

Git LFS initialized.


##### Basic Values/Constants

In [8]:
# 设置模型的检查点，这里使用的是 Google 提供的 Vision Transformer
#(ViT) 基础模型，图片大小为 224x224，预训练于 ImageNet21k 数据集。
MODEL_CKPT = 'google/vit-base-patch16-224-in21k'
# 设置训练的总轮数。
NUM_OF_EPOCHS = 10

# 设置学习率。
LEARNING_RATE = 2e-4
# 设置训练的步数。
STEPS = 100

# 设置批处理大小。
BATCH_SIZE = 16
# 设置设备，这里使用的是 Metal Performance Shaders (MPS) 设备，
#适用于在 Apple 设备上进行神经网络计算。
DEVICE = torch.device("cuda")

# 设置报告输出方式，这里使用 TensorBoard 进行可视化展示。
REPORTS_TO = 'tensorboard'

##### Load ViT Feature Extractor

In [9]:
'''使用 ViTFeatureExtractor.from_pretrained 方法从预训练的
Vision Transformer (ViT) 模型中加载特征提取器。MODEL_CKPT 是模型的检查点，
它指定了预训练模型的来源。在你之前的代码中，MODEL_CKPT 被设置为
'google/vit-base-patch16-224-in21k'，这意味着你将从 Hugging Face 
模型库中加载 Google 提供的 ViT 基础模型，它的图片大小为 224x224，
预训练于 ImageNet21k 数据集。

这个特征提取器用于将图像数据预处理成模型所需的格式。例如，
它会对图像进行大小调整、归一化等操作。你可以使用这个特征提取器来处理你的图像数据，
然后将处理后的数据输入到 ViT 模型中进行预测或训练。'''
from transformers import ViTImageProcessor
feature_extractor = ViTImageProcessor.from_pretrained(MODEL_CKPT)

#feature_extractor = ViTFeatureExtractor.from_pretrained(MODEL_CKPT)

##### Preprocessing Dataset

In [10]:
def transform(sample_batch):
    # 将一个包含 PIL 图像的列表转换为像素值
    # feature_extractor 是之前定义的用于提取特征的对象
    # 这里使用列表推导式 [x for x in sample_batch['image']] 获取图像列表
    # return_tensors="pt" 指定返回的张量类型为 PyTorch 张量
    inputs = feature_extractor([x.convert("RGB") for x in sample_batch['image']], return_tensors="pt")
    # 将一个包含 PIL 图像的列表转换为 RGB 格式的像素值
    
    # 准备标签
    # 将样本批次中的标签添加到输入字典中
    inputs['labels'] = sample_batch['label']
    
    # 返回处理后的输入字典，包含像素值和标签
    return inputs


##### Apply Transform Function to Dataset

In [11]:
# 使用之前定义的 transform 函数对数据集进行预处理
# dataset 是之前加载的数据集，包含图像和标签
# with_transform 方法将 transform 函数应用于数据集中的每个样本
# 返回一个新的数据集，其中的样本已经被转换为模型所需的格式
prepped_ds = dataset.with_transform(transform)


#### Training & Evaluation

##### Define Data Collator

In [12]:
def data_collator(batch):
    # 定义一个数据整理函数，用于将一个批次的样本组合成一个批次的数据
    return {
        # 'pixel_values' 键对应的值是一个张量，包含了批次中所有样本的像素值
        # 使用 torch.stack 将列表中的每个样本的 'pixel_values' 张量
        # 堆叠成一个新的张量
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        
        # 'labels' 键对应的值是一个张量，包含了批次中所有样本的标签
        # 使用 torch.tensor 将列表中的每个样本的标签转换成一个新的张量
        'labels': torch.tensor([x['labels'] for x in batch])
    }


##### Define Evaluation Metric

In [13]:
def compute_metrics(p):
    # 加载准确率评估指标
    accuracy_metric = evaluate.load("accuracy")
    # 计算并获取准确率
    accuracy = accuracy_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)['accuracy']
    
    # 加载 F1 分数评估指标
    f1_score_metric = evaluate.load("f1")
    # 计算并获取加权平均、微平均和宏平均的 F1 分数
    weighted_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["f1"]
    micro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')['f1']
    macro_f1_score = f1_score_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["f1"]
    
    # 加载召回率评估指标
    recall_metric = evaluate.load("recall")
    # 计算并获取加权平均、微平均和宏平均的召回率
    weighted_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["recall"]
    micro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')["recall"]
    macro_recall = recall_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["recall"]
    
    # 加载精确度评估指标
    precision_metric = evaluate.load("precision")
    # 计算并获取加权平均、微平均和宏平均的精确度
    weighted_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='weighted')["precision"]
    micro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='micro')["precision"]
    macro_precision = precision_metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids, average='macro')["precision"]
    
    # 返回所有评估指标的字典
    return {"accuracy": accuracy, 
            "Weighted F1": weighted_f1_score,
            "Micro F1": micro_f1_score,
            "Macro F1": macro_f1_score,
            "Weighted Recall": weighted_recall,
            "Micro Recall": micro_recall,
            "Macro Recall": macro_recall,
            "Weighted Precision": weighted_precision,
            "Micro Precision": micro_precision,
            "Macro Precision": macro_precision
            }


##### Load Pretrained Model

In [14]:
# 获取数据集中的标签名称列表
labels = dataset['train'].features['label'].names

# 从预训练的检查点加载 Vision Transformer (ViT) 模型用于图像分类
# MODEL_CKPT 是之前定义的模型检查点名称
# num_labels 指定了模型需要输出的标签数量
# id2label 和 label2id 分别提供了从标签索引到标签名称和从标签名称到标签索引的映射
model = ViTForImageClassification.from_pretrained(
    MODEL_CKPT,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
).to(DEVICE)  # 将模型移动到指定的设备上，例如 GPU 或 CPU
print(len(labels))

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


36


##### Define Training Arguments

In [15]:
args = TrainingArguments(
    output_dir=MODEL_CKPT + "vegetables_classification_3.0",  # 指定模型和训练日志的输出目录
    remove_unused_columns=False,  # 是否移除数据集中未使用的列
    num_train_epochs=NUM_OF_EPOCHS,  # 指定训练的轮数
    evaluation_strategy="epoch",  # 指定评估策略，每个 epoch 结束时进行评估
    save_strategy="epoch",  # 指定保存策略，每个 epoch 结束时保存模型
    per_device_train_batch_size=BATCH_SIZE,  # 指定每个设备的训练批次大小
    learning_rate=LEARNING_RATE,  # 指定学习率
    report_to=REPORTS_TO,  # 指定报告的输出方式，例如 "tensorboard"
    disable_tqdm=False,  # 是否禁用进度条
    load_best_model_at_end=True,  # 训练结束时是否加载最佳模型
    metric_for_best_model="Weighted F1",  # 用于选择最佳模型的评估指标
    logging_first_step=True,  # 是否在第一步记录日志
    hub_private_repo=False,  ## 是否使用私有仓库推送到 Hugging Face Hub
    push_to_hub=False  ## 是否推送模型到 Hugging Face Hub
)


##### Instantiate Trainer

In [16]:
trainer = Trainer(
    model=model,  # 你之前定义的模型
    args=args,  # 训练参数，之前通过 TrainingArguments 类定义
    data_collator=data_collator,  # 数据整理函数，用于将多个样本组合成一个批次
    compute_metrics=compute_metrics,  # 评估指标计算函数，用于在评估时计算指标
    train_dataset=prepped_ds['train'],  # 训练数据集，之前通过 dataset.with_transform(transform) 预处理得到
    eval_dataset=prepped_ds['test'],  # 测试数据集，用于评估模型性能
    tokenizer=feature_extractor,  # 特征提取器，用于数据预处理
)


/home/yuan314/anaconda3/envs/vit/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


##### Train Model

In [17]:
train_results = trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Weighted f1,Micro f1,Macro f1,Weighted recall,Micro recall,Macro recall,Weighted precision,Micro precision,Macro precision
1,3.620500,0.599933,0.883008,0.865571,0.883008,0.865781,0.883008,0.883008,0.883025,0.883724,0.883008,0.884047
2,3.620500,0.267817,0.924791,0.922698,0.924791,0.922566,0.924791,0.924791,0.924383,0.937987,0.924791,0.938159
3,0.838600,0.200774,0.933148,0.931522,0.933148,0.931365,0.933148,0.933148,0.932716,0.943474,0.933148,0.943631
4,0.838600,0.143789,0.955432,0.954122,0.955432,0.953903,0.955432,0.955432,0.954938,0.964140,0.955432,0.964239
5,0.838600,0.130252,0.963788,0.962987,0.963788,0.962742,0.963788,0.963788,0.963272,0.967855,0.963788,0.967944
6,0.128100,0.117727,0.966574,0.965578,0.966574,0.965326,0.966574,0.966574,0.966049,0.974158,0.966574,0.974229
7,0.128100,0.101958,0.969359,0.969622,0.969359,0.969359,0.969359,0.969359,0.968827,0.973309,0.969359,0.973383
8,0.039700,0.106312,0.969359,0.969137,0.969359,0.968876,0.969359,0.969359,0.968827,0.974580,0.969359,0.974650
9,0.039700,0.103100,0.969359,0.969137,0.969359,0.968876,0.969359,0.969359,0.968827,0.974580,0.969359,0.974650
10,0.039700,0.103168,0.969359,0.969137,0.969359,0.968876,0.969359,0.969359,0.968827,0.974580,0.969359,0.974650


/home/yuan314/anaconda3/envs/vit/lib/python3.9/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/yuan314/anaconda3/envs/vit/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuan314/anaconda3/envs/vit/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/yuan314/anaconda3/envs/vit/lib/python3.9/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency exp

In [18]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)#记录训练指标
trainer.save_metrics("train", train_results.metrics)#保存训练指标
trainer.save_state()#保存训练状态

***** train metrics *****
  epoch                    =         10.0
  total_flos               = 2248781788GF
  train_loss               =       0.2648
  train_runtime            =   0:29:45.07
  train_samples_per_second =        17.45
  train_steps_per_second   =        1.092


epoch: 表示训练经过的总轮数（epochs）。在这个例子中，训练进行了 3 个 epochs。

total_flos: 表示训练过程中执行的浮点运算总数，以浮点运算次数（FLOPs）计算。在这个例子中，总共执行了约 3753.28 万亿次浮点运算。

train_loss: 表示训练过程中的平均损失（loss）。在这个例子中，最终的平均损失是 0.0134。

train_runtime: 表示训练的总运行时间。在这个例子中，训练总共花费了 11 分 44.25 秒。

train_samples_per_second: 表示每秒处理的样本数。在这个例子中，每秒处理了约 63.898 个样本。

train_steps_per_second: 表示每秒执行的训练步骤（或批次）数。在这个例子中，每秒执行了约 3.996 步训练。

##### Evaluate Model

In [19]:
# 使用测试数据集对模型进行评估
metrics = trainer.evaluate(prepped_ds['test'])

# 将评估结果记录到日志中
trainer.log_metrics("eval", metrics)

# 将评估指标保存到文件中
trainer.save_metrics("eval", metrics)


***** eval metrics *****
  epoch                   =       10.0
  eval_Macro F1           =     0.9694
  eval_Macro Precision    =     0.9734
  eval_Macro Recall       =     0.9688
  eval_Micro F1           =     0.9694
  eval_Micro Precision    =     0.9694
  eval_Micro Recall       =     0.9694
  eval_Weighted F1        =     0.9696
  eval_Weighted Precision =     0.9733
  eval_Weighted Recall    =     0.9694
  eval_accuracy           =     0.9694
  eval_loss               =      0.102
  eval_runtime            = 0:00:30.88
  eval_samples_per_second =     11.624
  eval_steps_per_second   =      1.457


这些是你的模型在评估（测试）阶段的性能指标。下面是对每个指标的简要解释：

- `epoch`: 训练完成的轮数（epochs）。

- `eval_Macro F1`: 宏平均 F1 分数，计算每个类别的 F1 分数然后取平均。

- `eval_Macro Precision`: 宏平均精确度，计算每个类别的精确度然后取平均。

- `eval_Macro Recall`: 宏平均召回率，计算每个类别的召回率然后取平均。

- `eval_Micro F1`: 微平均 F1 分数，先计算总的真正例、假正例和假负例，然后计算 F1 分数。

- `eval_Micro Precision`: 微平均精确度，先计算总的真正例和假正例，然后计算精确度。

- `eval_Micro Recall`: 微平均召回率，先计算总的真正例和假负例，然后计算召回率。

- `eval_Weighted F1`: 加权平均 F1 分数，根据每个类别的样本数加权计算 F1 分数。

- `eval_Weighted Precision`: 加权平均精确度，根据每个类别的样本数加权计算精确度。

- `eval_Weighted Recall`: 加权平均召回率，根据每个类别的样本数加权计算召回率。

- `eval_accuracy`: 准确率，正确分类的样本数占总样本数的比例。

- `eval_loss`: 评估阶段的平均损失。

- `eval_runtime`: 评估阶段的运行时间。

- `eval_samples_per_second`: 每秒处理的样本数。

- `eval_steps_per_second`: 每秒执行的步骤（批次）数。

这些指标提供了模型在测试数据集上的性能概览，可以帮助你了解模型的泛化能力和预测准确性。在这个例子中，模型的表现非常好，几乎所有指标都接近 1.0。

##### Push Model to Hub (My Profile!)

In [20]:
kwargs = {
    "finetuned_from" : model.config._name_or_path,
    "tasks" : "image-classification",
    "tags" : ["image-classification"],
}

if args.push_to_hub:
    trainer.push_to_hub("All Dunn!!!")
else:
    trainer.create_model_card(**kwargs)

### Notes & Other Takeaways
****
- Wow, I have never had a model that perfect results!
- I am not sure that I would want to use the third epoch version of the project. It is perfect, but there is concern about overtraining.
- 哇，我从来没有一个模型能达到如此完美的效果！
- 我不确定我是否想使用该项目的第三个历元版本。这是完美的，但人们担心过度训练。
****

In [21]:
import cv2

image_path = 'test/xihongshi.png'
image = cv2.imread(image_path)

#转换颜色空间：OpenCV 默认加载图片的颜色空间是 BGR，
#而通常模型需要的输入颜色空间是 RGB，因此你需要进行转换：
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#预处理图片
inputs = feature_extractor(images=image, return_tensors="pt")

#进行预测：然后，你可以将预处理后的图片输入到模型中，进行预测：
with torch.no_grad():  # 确保不计算梯度，节省计算资源
    outputs = model(**inputs.to(DEVICE))  # 将输入数据移动到相同的设备上
    
#解析预测结果：模型的输出通常是一个 logits 或者 probabilities 的张量。
#你可以使用 torch.argmax 来获取最可能的类别索引，然后将其转换成类别名称：
pred_label_idx = torch.argmax(outputs.logits, dim=1).item()  # 获取最可能的类别索引
pred_label = labels[pred_label_idx]  # 将索引转换成类别名称

print(f'Predicted label: {pred_label}')



Predicted label: tomato


In [22]:
outputs.logits

tensor([[-0.0263, -0.7184, -0.6585, -0.3652,  0.0969, -0.6789, -0.0830,  0.1112,
          0.2340, -0.4124, -0.5306,  0.5624, -0.4493, -0.4274, -0.5315, -0.2337,
         -0.6679, -0.8307, -0.4627, -0.0928, -0.2848, -0.4195, -0.4754, -0.2980,
         -0.5075, -0.4875,  0.0463, -0.0182, -0.2832, -0.7199, -0.5597, -0.1545,
         -0.0285,  8.3290, -0.4389, -0.3395]], device='cuda:0')

In [23]:
!ls

fruit-and-vegetable-image-detection-vit.ipynb  new
fruit_vegetable_image_detection		       test
google					       Untitled.ipynb
image					       Vegetables_ViT2.ipynb
MiSans-Normal.ttf			       Vegetables_ViT.ipynb
mlruns


In [24]:
!ls test/

xiangjiao.png  xihongshi  xihongshi.png


In [40]:
import cv2
from transformers import ViTForImageClassification, ViTImageProcessor
import torch
from torch.nn.functional import softmax

# 加载模型
model_path = 'google/vit-base-patch16-224-in21kvegetables_classification_3.0/checkpoint-1950'
model = ViTForImageClassification.from_pretrained(model_path)
#载预训练的图像处理器
feature_extractor = ViTImageProcessor.from_pretrained(model_path)

# 加载图片
image_path = 'test/xiangjiao.png'
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# 预处理图片
inputs = feature_extractor(images=image, return_tensors="pt")

# 使用模型进行预测
with torch.no_grad():
    outputs = model(**inputs)

# 获取预测结果
pred_label_idx = torch.argmax(outputs.logits, dim=1).item()
pred_label = model.config.id2label[pred_label_idx]


# 假设 outputs 是模型的输出，包含 logits
logits = outputs.logits

# 使用 softmax 函数将 logits 转换为概率
probs = softmax(logits, dim=1)

# 获取每个类别的概率及其索引
prob_list = [(p.item(), idx) for idx, p in enumerate(probs[0])]

# 按概率从大到小排序
sorted_probs = sorted(prob_list, key=lambda x: x[0], reverse=True)

# 打印表头
print(f"   {'Label':<16}{'Probability':>12}")

# 打印排序后的概率及对应的标签索引
for prob, idx in sorted_probs:
    print(f"{idx:>3}: {model.config.id2label[idx]:<14}{prob:>12.8f}")


# 打印识别结果
print(f'Predicted label: {pred_label_idx}.{pred_label}')


   Label            Probability
  1: banana          0.99571556
 19: mango           0.00029658
 24: peas            0.00023871
 17: lemon           0.00017473
  2: beetroot        0.00015865
  3: bell pepper     0.00015862
  9: corn            0.00015647
 21: orange          0.00014761
 25: pineapple       0.00014714
 16: kiwi            0.00014189
 15: jalepeno        0.00013929
 28: raddish         0.00013864
 31: sweetcorn       0.00013553
  5: capsicum        0.00013224
 32: sweetpotato     0.00012650
 29: soy beans       0.00011763
 18: lettuce         0.00011504
 26: pomegranate     0.00011380
  0: apple           0.00011268
 22: paprika         0.00011084
 12: garlic          0.00010987
 13: ginger          0.00010980
  6: carrot          0.00010641
 20: onion           0.00009548
 23: pear            0.00009530
  7: cauliflower     0.00009394
 27: potato          0.00009240
 30: spinach         0.00009165
 35: watermelon      0.00008826
 10: cucumber        0.00008779
 14: gra

In [16]:
model.config.id2label
#pred_label_idx


{0: 'apple',
 1: 'banana',
 10: 'cucumber',
 11: 'eggplant',
 12: 'garlic',
 13: 'ginger',
 14: 'grapes',
 15: 'jalepeno',
 16: 'kiwi',
 17: 'lemon',
 18: 'lettuce',
 19: 'mango',
 2: 'beetroot',
 20: 'onion',
 21: 'orange',
 22: 'paprika',
 23: 'pear',
 24: 'peas',
 25: 'pineapple',
 26: 'pomegranate',
 27: 'potato',
 28: 'raddish',
 29: 'soy beans',
 3: 'bell pepper',
 30: 'spinach',
 31: 'sweetcorn',
 32: 'sweetpotato',
 33: 'tomato',
 34: 'turnip',
 35: 'watermelon',
 4: 'cabbage',
 5: 'capsicum',
 6: 'carrot',
 7: 'cauliflower',
 8: 'chilli pepper',
 9: 'corn'}

In [19]:
torch.argmax(outputs.logits, dim=1)


tensor([1])